In [145]:
import networkx as nx
import numpy as np
from scipy.special import comb

In [169]:
N = 500
D = 128
T = 1
k = 16
k_adj = 1.55 
P = D / (N - 1)

H = 100 
STOP = 0.25
START_MEM = 100
r_expected = 40 

In [170]:
G = nx.DiGraph()
G.add_nodes_from(np.arange(N))

In [166]:
x, y = np.meshgrid(np.arange(N), np.arange(N))
mask = x != y
x = x[mask]
y = y[mask]
pairs = np.stack((x, y), axis=1)

z = np.random.default_rng().geometric(p=P, size=((N*N)-N))
num_edges = (z == 1).sum()

print("Number of edges determined: ", num_edges)

Number of edges determined:  64078


In [167]:
index = np.random.default_rng().choice(pairs.shape[0], 
                                       size=int(num_edges), 
                                       replace=False)

In [168]:
p = pairs[index]
weights = (np.ones(p.shape[0]) * (T / (k_adj * k))).reshape(-1, 1)
G.add_weighted_edges_from(np.concatenate([p, weights], axis = 1), weight='weight')

In [115]:
memory_bank = []
for i in np.arange(0, START_MEM):
    memory_A = np.random.default_rng().choice(np.arange(0,N-1), size=r_expected)
    memory_bank.append(memory_A)

i, j = np.meshgrid(np.arange(len(memory_bank)), np.arange(len(memory_bank)))
mask = i != j
i = i[mask]
j = j[mask]
pairs = np.unique(np.sort(np.stack((i,j), axis = 1)), axis=0)
np.random.shuffle(pairs)

In [116]:
total_inters = 0
ind = 0
inst_inters = 0
inst_len = 0

In [117]:
def interference_check(memory_bank, a, b, memory_C):
    sum = 0
    for i in range(len(memory_bank)):
        if i != a and i != b: 
            inter = list(set(memory_C.tolist()) & set(memory_bank[i]))
            if len(inter) > len(memory_bank[i]) / 2:
                sum += 2
    return sum

In [136]:
def JOIN_shared_one_step(memory_bank, i, j, adjmat):
    memory_A = memory_bank[i]
    memory_B = memory_bank[j]

    state = np.zeros(adjmat.shape[0])
    state[memory_A] = 1
    state[memory_B] = 1
    
    fired = np.heaviside((adjmat @ state) - 1, 1)
    
    memory_C = np.nonzero(fired)[0]
    
    inter = interference_check(memory_bank, i, j, memory_C)
    memory_bank.append(memory_C)
    
    return inter, len(memory_C)

In [141]:
adjmat = nx.adjacency_matrix(G)

total_inters = 0
ind = 0
inst_inters = 0
inst_len = 0

for pair in pairs:
    ind += 1
    i = pair[0]
    j = pair[1]
    inter_flag, length = JOIN_shared_one_step(memory_bank, i, j, adjmat)
    inst_len += length
    if ind % H == 0:
        print("Memories: ", len(memory_bank))
        print("Instantaneous interference rate: ", inst_inters/H)
        print("Average interference rate: ", total_inters/len(memory_bank))
        print("Average size of memories created: ", inst_len/H, "\n\n")
        inst_inters = 0
        inst_len = 0
    if inter_flag > 0:
        total_inters += inter_flag
        inst_inters += inter_flag
        if total_inters/len(memory_bank) > STOP:
            print("Config: N=", N, " D=",D, " k=", k, " k_adj=", k_adj, " R=", 
                  r_expected, "START_MEM=", START_MEM)
            print("Halting memory formation at ", len(memory_bank), 
                  " memories due to more than ", STOP*100, 
                  "percent total interference")
            print("Instantaneous interference rate: ", inst_inters/H)
            print("Average interference rate: ", total_inters/len(memory_bank))
            break

C:\Users\Shosei\AppData\Local\Temp\ipykernel_27872\3491176581.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjmat = nx.adjacency_matrix(G)


Memories:  201
Instantaneous interference rate:  0.14
Average interference rate:  0.06965174129353234
Average size of memories created:  37.88 


Config: N= 500  D= 128  k= 16  k_adj= 1.55  R= 40 START_MEM= 100
Halting memory formation at  274  memories due to more than  25.0 percent total interference
Instantaneous interference rate:  0.56
Average interference rate:  0.25547445255474455
